# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846762731669                   -0.70    4.2         
  2   -7.852301678891       -2.26       -1.53    1.0   23.0ms
  3   -7.852613300301       -3.51       -2.55    1.5   25.9ms
  4   -7.852645837523       -4.49       -2.88    2.8   32.2ms
  5   -7.852646483600       -6.19       -3.18    1.0   23.1ms
  6   -7.852646677971       -6.71       -4.04    1.0   23.1ms
  7   -7.852646686541       -8.07       -5.15    1.8   28.4ms
  8   -7.852646686721       -9.74       -5.35    2.2   29.7ms
  9   -7.852646686729      -11.10       -6.14    1.0   83.4ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846788626083                   -0.70           4.5         
  2   -7.852526668833       -2.24       -1.64   0.80    2.2    263ms
  3   -7.852636566437       -3.96       -2.72   0.80    1.0   21.9ms
  4   -7.852646441168       -5.01       -3.27   0.80    2.2   28.6ms
  5   -7.852646672195       -6.64       -4.10   0.80    1.5   24.1ms
  6   -7.852646686300       -7.85       -4.75   0.80    1.8   26.3ms
  7   -7.852646686723       -9.37       -5.62   0.80    1.5   24.7ms
  8   -7.852646686730      -11.20       -6.63   0.80    1.8   25.8ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.386589e+01     3.695087e+00
 * time: 0.05808520317077637
     1     1.109922e+00     2.038859e+00
 * time: 0.283217191696167
     2    -1.498724e+00     2.427578e+00
 * time: 0.306380033493042
     3    -3.720102e+00     1.988739e+00
 * time: 0.3397030830383301
     4    -4.955416e+00     1.638416e+00
 * time: 0.3725261688232422
     5    -6.657182e+00     9.513314e-01
 * time: 0.40588808059692383
     6    -7.297571e+00     4.401271e-01
 * time: 0.4389631748199463
     7    -7.563065e+00     1.741518e-01
 * time: 0.4620211124420166
     8    -7.695584e+00     1.541590e-01
 * time: 0.4850611686706543
     9    -7.776845e+00     9.976182e-02
 * time: 0.5078170299530029
    10    -7.816598e+00     5.327630e-02
 * time: 0.5308201313018799
    11    -7.838248e+00     5.049022e-02
 * time: 0.5538370609283447
    12    -7.844896e+00     4.989172e-02
 * time: 0.5768611431121826
    13    -7.846990e+00     3.725981e-02
 * time: 0.5998971462

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846907696350                   -0.70    4.8         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645943299                   -1.64         
  2   -7.852646686730       -6.13       -3.72    1.93s
  3   -7.852646686730      -13.36       -7.27    181ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 5.684610343332681e-7
|ρ_newton - ρ_scfv| = 2.0944093941458112e-7
|ρ_newton - ρ_dm|   = 7.391852524326958e-10
